## Generation of log files
To run this code make sure that both REVOL-E-TION and LIFT are installed in your current Python environment

In [1]:
from pathlib import Path
import pandas as pd
import simpy

from lift.definitions import SUBFLEETS
from revoletion import mobility as mob
from revoletion import dispatch as dis

C:\Users\ga42hum\AppData\Local\anaconda3\envs\lift\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
cwd = Path.cwd()
dti = pd.date_range(start='1/1/2023',
                    end='01/01/2024',
                    freq='1h',
                    tz='Europe/Berlin')

To change the number of generated trips per day you have to adapt the timeframemapper in mtf.py

In [3]:
def generate_demand(subfleet, num=50, p_chg=100000):
    demand = mob.VehicleDemand(dti=dti)
    demand.from_usecases(path_usecases=cwd / 'def_mobility' / f'usecases_{subfleet.name}.csv',
                         path_timeframe_mapper=cwd / 'def_mobility' / 'mtf.py',
                         path_demand=None,
                         key_timeframe_mapper='bev')

    dispatcher = dis.GroupDispatcher(
        timer=dis.DispatchTimer(dti_base=dti),
        demand=demand,
        env=simpy.Environment(),
        factor_derate=0.9,
        params=dis.SubFleetParams(
            name=subfleet.name,
            units=[f'{subfleet.name}{i}' for i in range(num)],
            size_unit=subfleet.battery_capactiy_wh,
            pwr_chg=p_chg,
            is_electric=True,
            soc_upper=0.95,
            soc_lower=0.05,
            eff_chg=0.95,
            eff_roundtrip=0.95,
            loss_rate_per_hour=0
        ),
    )
    dispatcher.run_standalone()
    dispatcher.save_data(path_log=f'log_{subfleet.name}.csv')  # optional

In [4]:
for sf in SUBFLEETS.values():
    generate_demand(subfleet=sf)